### Necessary Imports

In [9]:
from PIL import Image
import pytesseract
from pytesseract import Output
import cv2
from pdf2image import convert_from_path
import os
import glob
import json

### Pdf To Image Conversion

In [10]:
pytesseract.pytesseract.tesseract_cmd=r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
images = convert_from_path("Sanskrit_Text.pdf",65)
path = 'C:/Users/tejas/OCR/'

In [11]:
for i in range(len(images)):   
      # Save pages as images in the pdf
    images[i].save(os.path.join(path ,'page'+ str(i) +'.jpg'), 'JPEG')

### Saving Images In List Format

In [12]:
img_list = list(map(cv2.imread,glob.glob('{}*.jpg'.format(path))))

### Dumping Data To Json File

In [13]:
def write_json(data, filename='C:/Users/tejas/OCR_BOXES/box_coordinates.json'):
    with open(filename,'w') as f:
        json.dump(data, f, indent=4)

### Appending Data To Json File

In [14]:
def box_details(name,tl,tr,bl,br):
    with open('C:/Users/tejas/OCR_BOXES/box_coordinates.json', mode='r', encoding='utf-8') as jf:
        data = json.load(jf)
        temp = data['Bounding_Boxes']
        # python object to be appended
        y = {'name': name, 'top_left': tl ,'top_right': tr ,'bottom_left': bl , 'bottom_right': br}
        # appending data to 'Bounding_Boxes'
        temp.append(y)
    write_json(data)

### Code For Generating Bounding Boxes

In [15]:
bar = progressbar.ProgressBar(maxval=len(members)).start()
counter = 1
for i in range(len(img_list)):
    img = img_list[i]
    hImg,wImg,_ = img.shape
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    original = img.copy()
    boxes = pytesseract.image_to_data(img, lang = 'hin')
    for x,b in enumerate(boxes.splitlines()):
        ROI_number = x
        if x!=0 :
            b = b.split()
            if len(b) == 12:
                x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
                cv2.rectangle(img,(x,y),(w+x,h+y),(0,0,255),1)
                ROI = original[y:y+h, x:x+w]
                cv2.imwrite('C:/Users/tejas/OCR_BOXES/box_imgs/box_{}_{}.png'.format(counter,ROI_number), ROI)
                name = 'box_{}_{}'.format(counter,ROI_number)
                tl = '[{},{}]'.format(x , y)
                tr = '[{},{}]'.format(x+w , y)
                bl = '[{},{}]'.format(x , y+h)
                br = '[{},{}]'.format(x+w , y+h)
                box_details(name,tl,tr,bl,br)
    counter = counter + 1